# Pulldown assay data processing: LY6A-Fc, LY6C1-Fc

`sequence`: variant DNA sequence  
`AA_sequence`: variant 7mer amino acid sequence

**Data samples and replicates**: 
- starter-virus (starter, 3 replicates)
    - `starter_virus_1_counts`
    - `starter_virus_2_counts`
    - `starter_virus_3_counts`  
- LY6A-Fc (3 replicates)
    - `LY6A_1_counts`
    - `LY6A_2_counts`
    - `LY6A_3_counts`  
- LY6C1-Fc (3 replicates)
    - `LY6C1_1_counts`
    - `LY6C1_2_counts`
    - `LY6C1_3_counts`  



## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os, sys

# import matplotlib as mpl
# import matplotlib.pyplot as plt
# import seaborn as sns

from pathlib import Path
current_dir = Path(os.getcwd())
top_dir = current_dir.parent
sys.path.append(str(top_dir))

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from utils.data_processing import *
from figures.utils.fig_utils import download_data, urls

## Load data

In [ ]:
download_data('../data/pulldown_assay_counts.csv', urls['pulldown_assay_counts.csv'])

In [ ]:
df = pd.read_csv('../data/pulldown_assay_counts.csv')
df

In [5]:
list(df.columns)

['sequence',
 'AA_sequence',
 'starter_virus_1_counts',
 'starter_virus_2_counts',
 'starter_virus_3_counts',
 'LY6A_1_counts',
 'LY6A_2_counts',
 'LY6A_3_counts',
 'LY6C1_1_counts',
 'LY6C1_2_counts',
 'LY6C1_3_counts']

In [6]:
# Since the dataframe contains read count values, a NaN value corresponds to 0 reads
df = df.fillna(0)

## RPM
First we need to compute the reads per million (RPM) of each column.

In [7]:
# Isolate the columns corresponding to read counts
counts_cols = [col for col in df.columns if "counts" in col]
counts_cols

['starter_virus_1_counts',
 'starter_virus_2_counts',
 'starter_virus_3_counts',
 'LY6A_1_counts',
 'LY6A_2_counts',
 'LY6A_3_counts',
 'LY6C1_1_counts',
 'LY6C1_2_counts',
 'LY6C1_3_counts']

In [8]:
# Create a new RPM column for each counts column 
RPM_cols = [re.sub("_counts", "_RPM", col) for col in counts_cols]
RPM_cols

['starter_virus_1_RPM',
 'starter_virus_2_RPM',
 'starter_virus_3_RPM',
 'LY6A_1_RPM',
 'LY6A_2_RPM',
 'LY6A_3_RPM',
 'LY6C1_1_RPM',
 'LY6C1_2_RPM',
 'LY6C1_3_RPM']

In [9]:
# Compute reads per million (RPM) and store in a new dataframe with renamed columns
df_RPM = df.drop(columns=counts_cols)
RPM_values = RPM(df[counts_cols]).rename(columns=dict(zip(counts_cols, RPM_cols)))
df_RPM = pd.concat([df_RPM, RPM_values], axis=1)
df_RPM

,sequence,AA_sequence,starter_virus_1_RPM,starter_virus_2_RPM,starter_virus_3_RPM,LY6A_1_RPM,LY6A_2_RPM,LY6A_3_RPM,LY6C1_1_RPM,LY6C1_2_RPM,LY6C1_3_RPM
0,AATGCTGTGAGTCGGGCTGGG,NAVSRAG,329.700380,334.384942,336.334333,21.424134,50.797720,36.544493,90.000646,101.929615,110.859467
1,GTGCGGCCTAATGGGTCTGGT,VRPNGSG,326.575902,330.955353,327.792104,38.303754,50.239503,30.219485,133.822949,165.999659,177.375148
2,GGTACGCGGCCTGGGCTTTTG,GTRPGLL,246.562089,235.982115,235.867519,4.544513,23.445101,15.461132,19.319510,9.319279,27.388810
3,TTTGGGGGTTCGCGGACTCCG,FGGSRTP,215.045613,214.745044,217.890590,30.513160,40.749819,17.569468,65.026644,75.136688,110.859467
4,TTGGATAAGAGGAATCTTGTT,LDKRNLV,206.215565,192.188900,207.690913,16.879620,8.931467,16.866689,42.408681,32.617477,48.908589
...,...,...,...,...,...,...,...,...,...,...,...
1750113,GCTGCTAGGGCTGCTACTGGT,AARAATG,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1750114,GGGGTGCCGGGTTATGCTTGG,GVPGYAW,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1750115,GGTAATTATCTTCATGCTTTG,GNYLHAL,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1750116,CGTGAGCCTAGTCCTAGTAGG,REPSPSR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Aggregating sample replicates
Each sample has 3 replicates, labeled `1`, `2` and `3`. We want the mean RPM, as well
as coefficient of variation (CV, computed as *standard deviation / mean*), of the 3
replicates per sample.

**Note:** if there are variants with $RPM=0$ across all replicates, then `mean_RPM` $=0$,
which results in 0-division when computing CV, and yields a `NaN` value. In this case,
it's not such a big deal because we end up dropping these variants from the SVAE
training data anyway.

In [10]:
# First get the names of the samples
samples = pd.unique([re.sub('_([123])_RPM', '', col) for col in RPM_cols])
samples

array(['starter_virus', 'LY6A', 'LY6C1'], dtype=object)

In [11]:
# Compute mean and CV (coefficient of variation) RPM, and store in a new dataframe
df_mean_cv_RPM = df.drop(columns=counts_cols)
for sample in samples:
    replicate_cols = [col for col in df_RPM.columns if sample in col]
    mean_RPM = df_RPM[replicate_cols].mean(axis=1)
    cv_RPM = df_RPM[replicate_cols].std(axis=1) / mean_RPM
    df_mean_cv_RPM["{}_mean_RPM".format(sample)] = mean_RPM
    df_mean_cv_RPM["{}_cv_RPM".format(sample)] = cv_RPM
df_mean_cv_RPM


,sequence,AA_sequence,starter_virus_mean_RPM,starter_virus_cv_RPM,LY6A_mean_RPM,LY6A_cv_RPM,LY6C1_mean_RPM,LY6C1_cv_RPM
0,AATGCTGTGAGTCGGGCTGGG,NAVSRAG,333.473218,0.010225,36.255449,0.405151,100.929909,0.103689
1,GTGCGGCCTAATGGGTCTGGT,VRPNGSG,328.441120,0.006883,39.587581,0.254412,159.065919,0.142009
2,GGTACGCGGCCTGGGCTTTTG,GTRPGLL,239.470574,0.025647,14.483582,0.655096,18.675866,0.484687
3,TTTGGGGGTTCGCGGACTCCG,FGGSRTP,215.893749,0.008040,29.610816,0.392306,83.674266,0.287778
4,TTGGATAAGAGGAATCTTGTT,LDKRNLV,202.031793,0.042350,14.225926,0.322309,41.311582,0.198510
...,...,...,...,...,...,...,...,...
1750113,GCTGCTAGGGCTGCTACTGGT,AARAATG,0.000000,NaN,0.000000,NaN,0.000000,NaN
1750114,GGGGTGCCGGGTTATGCTTGG,GVPGYAW,0.000000,NaN,0.000000,NaN,0.000000,NaN
1750115,GGTAATTATCTTCATGCTTTG,GNYLHAL,0.000000,NaN,0.000000,NaN,0.000000,NaN
1750116,CGTGAGCCTAGTCCTAGTAGG,REPSPSR,0.000000,NaN,0.000000,NaN,0.000000,NaN


## (Log2) enrichment
Now, as a way to measure the performance of each variant in each assay, we want to
compute the assay enrichment (fold change of mean RPM) relative to the `starter_virus` sample. We
take the $\log_2$ of enrichment to keep all of the values on a similar scale.

**Note:** like when computing CV above, variants with `mean_RPM` $=0$ can cause some
problems: 
- when `starter_virus_mean_RPM` $=0$, computing enrichment results in
0-division
- when the enrichment value exists (`starter_virus_mean_RPM` $\neq 0$) but is 0 (which
  happens when `<assay>_mean_RPM` $=0$), then taking the $\log_2$ results in `log2enr`
  $= -\infty$ (`-np.inf`).

Again, this isn't a big deal in this case because those variants will be ultimately
dropped anyway.

In [12]:
# Compute enrichment and log2 enrichment, and store in a new dataframe
df_enr = df.drop(columns=counts_cols)
for sample in samples[1:]: # We only do this for non-starter-virus samples
    enr = df_mean_cv_RPM["{}_mean_RPM".format(sample)]/df_mean_cv_RPM["starter_virus_mean_RPM"]
    log2enr = np.log2(enr)
    df_enr["{}_enr".format(sample)] = enr
    df_enr["{}_log2enr".format(sample)] = log2enr
df_enr


/usr/local/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


,sequence,AA_sequence,LY6A_enr,LY6A_log2enr,LY6C1_enr,LY6C1_log2enr
0,AATGCTGTGAGTCGGGCTGGG,NAVSRAG,0.108721,-3.201301,0.302663,-1.724217
1,GTGCGGCCTAATGGGTCTGGT,VRPNGSG,0.120532,-3.052515,0.484306,-1.046010
2,GGTACGCGGCCTGGGCTTTTG,GTRPGLL,0.060482,-4.047358,0.077988,-3.680601
3,TTTGGGGGTTCGCGGACTCCG,FGGSRTP,0.137155,-2.866125,0.387572,-1.367466
4,TTGGATAAGAGGAATCTTGTT,LDKRNLV,0.070414,-3.827988,0.204481,-2.289964
...,...,...,...,...,...,...
1750113,GCTGCTAGGGCTGCTACTGGT,AARAATG,NaN,NaN,NaN,NaN
1750114,GGGGTGCCGGGTTATGCTTGG,GVPGYAW,NaN,NaN,NaN,NaN
1750115,GGTAATTATCTTCATGCTTTG,GNYLHAL,NaN,NaN,NaN,NaN
1750116,CGTGAGCCTAGTCCTAGTAGG,REPSPSR,NaN,NaN,NaN,NaN


## Export processed data

Now we'll write out the `log2enr` and `cv_RPM` values of the `LY6A`-Fc and `LY6C1`-Fc pulldown
assays, along with the 7mer `AA_sequence`s and `mean_RPM` values for all samples
(including the starter)
to a .csv file, so we can use them to train the SVAE. 

In [13]:
df_for_SVAE = df_mean_cv_RPM[['AA_sequence', 'starter_virus_mean_RPM']].copy()
df_for_SVAE['LY6A_log2enr'] = df_enr['LY6A_log2enr']
df_for_SVAE['LY6A_cv_RPM'] = df_mean_cv_RPM['LY6A_cv_RPM']
df_for_SVAE['LY6A_mean_RPM'] = df_mean_cv_RPM['LY6A_mean_RPM']
df_for_SVAE['LY6C1_log2enr'] = df_enr['LY6C1_log2enr']
df_for_SVAE['LY6C1_cv_RPM'] = df_mean_cv_RPM['LY6C1_cv_RPM']
df_for_SVAE['LY6C1_mean_RPM'] = df_mean_cv_RPM['LY6C1_mean_RPM']
df_for_SVAE

,AA_sequence,starter_virus_mean_RPM,LY6A_log2enr,LY6A_cv_RPM,LY6A_mean_RPM,LY6C1_log2enr,LY6C1_cv_RPM,LY6C1_mean_RPM
0,NAVSRAG,333.473218,-3.201301,0.405151,36.255449,-1.724217,0.103689,100.929909
1,VRPNGSG,328.441120,-3.052515,0.254412,39.587581,-1.046010,0.142009,159.065919
2,GTRPGLL,239.470574,-4.047358,0.655096,14.483582,-3.680601,0.484687,18.675866
3,FGGSRTP,215.893749,-2.866125,0.392306,29.610816,-1.367466,0.287778,83.674266
4,LDKRNLV,202.031793,-3.827988,0.322309,14.225926,-2.289964,0.198510,41.311582
...,...,...,...,...,...,...,...,...
1750113,AARAATG,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000
1750114,GVPGYAW,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000
1750115,GNYLHAL,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000
1750116,REPSPSR,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000


In [14]:
df_for_SVAE.to_csv("../data/pulldown_assay_SVAE_data.csv", index=False)